In [232]:
import numpy as np
import heapq
from collections import Counter

In [233]:
with open('input16.txt', "r") as file:
        text = file.read().strip().split('\n')

# text = '''###############
# #.......#....E#
# #.#.###.#.###.#
# #.....#.#...#.#
# #.###.#####.#.#
# #.#.#.......#.#
# #.#.#####.###.#
# #...........#.#
# ###.#.#####.#.#
# #...#.....#.#.#
# #.#.#.###.#.#.#
# #.....#...#.#.#
# #.###.#.#.#.#.#
# #S..#.....#...#
# ###############'''.strip().split()
# text = '''#################
# #...#...#...#..E#
# #.#.#.#.#.#.#.#.#
# #.#.#.#...#...#.#
# #.#.#.#.###.#.#.#
# #...#.#.#.....#.#
# #.#.#.#.#.#####.#
# #.#...#.#.#.....#
# #.#.#####.#.###.#
# #.#.#.......#...#
# #.#.###.#####.###
# #.#.#...#.....#.#
# #.#.#.#####.###.#
# #.#.#.........#.#
# #.#.#.#########.#
# #S#.............#
# #################'''.strip().split('\n')
text = np.array([list(x) for x in text])
x, y = np.where(text == 'S')
start = x[0], y[0]
text[start] = '.'
x, y  = np.where(text == 'E')
end = x[0], y[0]
text[end] = '.'

In [234]:
def turning_cost(x, y, nx, ny, direction):
    if nx > x and y == ny:
        value = 1
    elif nx < x and ny == y:
        value = 3
    elif ny > y and nx == x:
        value = 0
    elif ny < y and nx == x:
        value = 2
    return ((abs(direction - value) % 3) + (abs(direction - value) == 3) ) * 1000, value

In [235]:
def dijkstra(maze, start, end, facing = 0):
    rows, cols = len(maze), len(maze[0])
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    def is_valid(x, y):
        return 0 <= x < rows and 0 <= y < cols and maze[x][y] == '.'

    pq = [(0, start[0], start[1], facing)]
    distances = {(start[0], start[1]): 0}

    while pq:
        cost, x, y, facing = heapq.heappop(pq)
        if (x, y) == end:
            return cost, distances
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid(nx, ny):
                tc, new_facing = turning_cost(x, y, nx, ny, facing)
                new_cost = cost + 1 + tc
                if (nx, ny) not in distances or new_cost < distances[(nx, ny)]:
                    distances[(nx, ny)] = new_cost
                    heapq.heappush(pq, (new_cost, nx, ny, new_facing))
    return -1

maze = text
result, distances = dijkstra(maze, start, end, facing=0)
result

83432

In [236]:
def find_paths(grid, start, end, facing=0):
    rows, cols = len(grid), len(grid[0])
    paths = []
    cost = 0 

    def is_valid(x, y, path):
        return 0 <= x < rows and 0 <= y < cols and (x, y) not in path and grid[x][y] == '.'

    def dfs(x, y, path, cost, facing):
        if (x, y) == end:
            paths.append([cost, path[:]])
            return
        path.append((x, y))

        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nx, ny = x + dx, y + dy
            if is_valid(nx, ny, path):
                tc, new_facing = turning_cost(x, y, nx, ny, facing)
                new_cost = cost + tc + 1

                if distances.get((nx, ny)) is not None:
                    if distances[(nx, ny)] in [new_cost, new_cost-1000]:
                        dfs(nx, ny, path, new_cost, new_facing)

        path.pop()
    dfs(start[0], start[1], [], cost, facing)
    return paths

# Example usage
grid = text

all_paths = find_paths(grid, start, end, facing=0)
min_score = min([x[0] for x in all_paths])
points = []
for path in all_paths:
    if (path[0]) == min_score:
        points.extend(path[1])
len(set(points)) + 1 # HOTFIX: why +1?

467